In [1]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import dialogflow
updater = Updater(token='1156128317:AAGZxr2B7G8SWPtYqcYLBTwyh8valJrLAK4') 
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'vknlp-bot-npi9-83224a45e4ad.json'# скачнный JSON

DIALOGFLOW_PROJECT_ID = 'vknlp-bot-npi9' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'VKNLP_bot'  # ID бота из телеграма

D:\Prog\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  


In [6]:
#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w", encoding="utf-8") as fout:
    with open("Otvety.txt", "r", encoding="utf-8") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [24]:
def identity_tokenizer(text):
    return text

In [7]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [37]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(200)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]]
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [10]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r", encoding="utf-8") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

In [18]:
sentences = [i for i in sentences if len(i) > 2]

In [19]:
min([len(tokens) for tokens in sentences])

3

In [20]:
max([len(tokens) for tokens in sentences])

5274

In [15]:
np.mean([len(tokens) for tokens in sentences])

12.01168988310117

In [16]:
np.median([len(tokens) for tokens in sentences])

4.0

In [21]:
sentences = [tokens[:min(100, len(tokens) - 1)] for tokens in sentences]

In [22]:
max([len(tokens) for tokens in sentences])

100

In [25]:
modelFT = FastText(sentences=sentences, size=100, min_count=2, window=5)

In [26]:
modelFT.save("ft_model")

In [29]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer,lowercase=False, max_df=0.9)

In [30]:
tfidf_vectorizer.fit_transform(sentences)

<64853x123820 sparse matrix of type '<class 'numpy.float64'>'
	with 739258 stored elements in Compressed Sparse Row format>

In [34]:
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r", encoding="utf-8") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

ft_index.build(100)

True

In [36]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Добрый день')

In [ ]:
def textMessage(bot, update):

    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
    text_input = dialogflow.types.TextInput(text=update.message.text , language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.types.QueryInput(text=text_input)
    try:
        response = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
         raise

    text = response.query_result.fulfillment_text
    if text:
        bot.send_message(chat_id=update.message.chat_id, text= response.query_result.fulfillment_text)
    else:
        custom_response = get_response(update.message.text, ft_index, modelFT, index_map)
        bot.send_message(chat_id=update.message.chat_id, text=custom_response[0])

In [ ]:
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
# Добавляем хендлеры в диспетчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота, если были нажаты Ctrl + C
updater.idle()